We can improve the Fermat test by using Euler’s theorem. If $p$ is an odd prime then
\begin{equation}
    a^{(p−1)/2} \equiv \left(\frac{a}{p}\right) \mod{p}
\end{equation}
where $\left(\frac{a}{p}\right)$ is the Legendre symbol. The Euler test base $a$, for an odd integer $N$, is to compute
\begin{equation}
    a^{(N−1)/2} \mod{N}
\end{equation}
and check if this is $\pm 1$ and equal to the Jacobi symbol $\left(\frac{a}{N}\right)$. For $N$ odd and positive this satisfies the properties:
\begin{align}
    \left(\frac{a}{N}\right) &= \left(\frac{a \bmod N}{N}\right), \\
    \left(\frac{ab}{N}\right) &= \left(\frac{a}{N}\right)\left(\frac{b}{N}\right), \\
    \left(\frac{-1}{N}\right) &= (-1)^{(N-1)/2}, \\
    \left(\frac{2}{N}\right) &= (-1)^{(N^2-1)/8},
\end{align}
and if $M$ and $N$ are odd and coprime,
\begin{equation}
    \left(\frac{M}{N}\right)\left(\frac{N}{M}\right) = (-1)^{(M-1)(N-1)/4}.
\end{equation}
If N is prime then $\left(\frac{a}{N}\right)$ is just the Legendre symbol. An Euler pseudoprime base $a$ is a composite $N$ which passes the Euler test base $a$, and an absolute Euler pseudoprime is a composite number which passes the Euler test for any base $a$ with $(a, N) = 1$.

In [ ]:
import math

def is_prime(n):
    '''
    A number n is prime if it is not divisible by any integer t with 1 < t <= sqrt(n).
    '''
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def modular_pow(base, exponent, modulus):
    '''
    Calculates (base^exponent) % modulus efficiently.
    '''
    result = 1
    base %= modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def jacobi_symbol(a, n):
    '''
    Calculates the Jacobi symbol (a/n) using a standard iterative algorithm.
    n must be a positive, odd integer.
    '''
    if n <= 0 or n % 2 == 0:
        raise ValueError("Jacobi symbol is only defined for positive odd n.")
    t = 1
    a %= n
    t = 1
    while a != 0:
        # Rule: Factor out all powers of 2 from 'a'
        while a % 2 == 0:
            a //= 2
            n_mod_8 = n % 8
            # Rule (2/n) = (-1)^((n^2-1)/8)
            if n_mod_8 in (3, 5):
                t = -t  # Flip the sign
        # At this point, 'a' is guaranteed to be odd.
        # Rule: Quadratic Reciprocity
        # Swap 'a' and 'n'
        a, n = n, a
        # Apply sign change if a and n were both congruent to 3 (mod 4)
        if a % 4 == 3 and n % 4 == 3:
            t = -t
        # Rule: (a/n) = (a mod n / n) for the next iteration
        a %= n
    if n == 1:
        return t
    else:  # This case occurs if the original gcd(a, n) was > 1
        return 0

In [ ]:
def euler_test(N, a):
    '''
    Performs the Euler primality test for an odd number N with base a.
    The test passes if a^((N-1)/2) is congruent to the Jacobi symbol (a/N) mod N.
    '''
    if N <= 1 or N % 2 == 0:
        return False
    jacobi = jacobi_symbol(a, N)
    # If gcd(a, N) > 1, jacobi is 0. This is a witness to compositeness.
    if jacobi == 0:
        return False
    # Calculate the left-hand side
    lhs = modular_pow(a, (N - 1) // 2, N)
    # Calculate the right-hand side
    rhs = N - 1 if jacobi == -1 else 1

    return lhs == rhs

def run_euler_on_intervals(start, end, max_a):
    '''
    Runs the Euler test on an interval for bases up to base a.
    '''
    euler_pseudoprimes = []
    prime_numbers = []
    for n in range(start, end + 1):
        if n <= 1:
            continue
        is_probable_prime = True
        for a in range(2, max_a + 1):
            if not euler_test(n, a):
                is_probable_prime = False
                break
        if is_probable_prime:
            if not is_prime(n):
                euler_pseudoprimes.append(n)
            else:
                prime_numbers.append(n)
    return euler_pseudoprimes, prime_numbers

# Define the intervals and the maximum base 'a'
start1, end1 = 188000, 188200
start2, end2 = 10**9, 10**9 + 200
a_limit = 20

# Run the test on the first interval
print(f"Euler test on the interval [{start1}, {end1}]:")
numbers_1 = run_euler_on_intervals(start1, end1, a_limit)
print(f"Euler pseudoprimes: {numbers_1[0]}")
print(f"Prime numbers: {numbers_1[1]}")

# Run the test on the second interval
print(f"\nEuler test on the interval [{start2}, {end2}]:")
numbers_2 = run_euler_on_intervals(start2, end2, a_limit)
print(f"Euler pseudoprimes: {numbers_2[0]}")
print(f"Prime numbers: {numbers_2[1]}")

Euler test on the interval [188000, 188200]:
Euler pseudoprimes: []
Prime numbers: [188011, 188017, 188021, 188029, 188107, 188137, 188143, 188147, 188159, 188171, 188179, 188189, 188197]

Euler test on the interval [1000000000, 1000000200]:
Euler pseudoprimes: []
Prime numbers: [1000000007, 1000000009, 1000000021, 1000000033, 1000000087, 1000000093, 1000000097, 1000000103, 1000000123, 1000000181]


We note that there are no absolute Euler pseudoprimes. This differs from most literature which only tests Euler's criterion up to $\pm 1$, not distinguishing between both answers. However, the minimum base we need to test now is $29$. This is because the Euler test is now seeing the Carmicheal numbers that were an absolute blind spot for the Fermat test.

In [ ]:
def find_absolute_euler_pseudoprimes(start, end):
    '''
    Absolute Euler pseudoprimes are composite numbers N
    which pass Eulers's test for all bases a coprime to N.
    '''
    absolute_euler_pseudoprimes = []
    for N in range(start, end + 1):
        if is_prime(N):
            continue
        is_absolute_euler_pseudoprime = True
        for a in range(2, 30):
            if math.gcd(a, N) == 1:
                if not euler_test(N, a):
                    is_absolute_euler_pseudoprime = False
                    break
        if is_absolute_euler_pseudoprime:
            absolute_euler_pseudoprimes.append(N)
    return absolute_euler_pseudoprimes

absolute_euler_pseudoprimes = find_absolute_euler_pseudoprimes(2, 10**6)
print(f"Absolute Euler pseudoprimes: {absolute_euler_pseudoprimes}")

Absolute Euler pseudoprimes: []
